In [1]:
import parsl
from parsl.app.app import sandbox_app, python_app
from parsl.data_provider.files import File
import os

print(parsl.version.VERSION)

1.1.0a1


In [2]:
parsl.load()
project="helloSandbox"

In [3]:
# The first sandbox_app()
# this app generates a semi-random number ( between 0 and 32,767 ) and saves it to a file.

@sandbox_app()
def generate(project=project):
    return "echo $(( RANDOM )) &> out.txt"


In [4]:
#The second sandbox_app()
#this app concatenates input files into a single output file
#to the app, it is necessary to pass only the workflow_schema because this is solved automatically
@sandbox_app()
def concat(inputs=[],project=project):
    return "cat {0} > out.txt".format(" ".join([i for i in inputs]))

In [5]:
# App that calculates the sum of values in a list of input files
@python_app
def total(inputs=[]):
    total = 0
    with open(inputs[0], 'r') as f:
        for l in f:
            total += int(l)
    return total

In [6]:
output_files = []

for i in range (2):
    output_files.append(generate())

In [7]:
# Concatenate the files into a single file passing workflow_schema

cc = concat(inputs=[i.workflow_schema.replace("workflow://","")+"/out.txt" for i in output_files])

In [9]:
# Calculate the sum of the random numbers
total = total(inputs=[File(project+"/"+cc.workflow_schema.replace("workflow://","")+"/out.txt")])
print (total.result())


42587
